# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")



run your solution on all test_images and make copies into the test_images directory).

In [ ]:
import scipy.spatial.distance as dist
import scipy.cluster.hierarchy as hier
import math

def print_image(image, grayscale):
    if (grayscale):
        plt.imshow(image, cmap='gray')
    else:
        plt.imshow(image)

def get_hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    return cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)

def pre_hough(image):
    gray = grayscale(image)
    smoothed = gaussian_blur(gray, 5)
    cannied = canny(smoothed, 50, 150)

    # Narrow image to region of interest.
    skyFraction = 5/9
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),
                          (imshape[1]/2, imshape[0] * skyFraction), 
                          (imshape[1]/2, imshape[0] * skyFraction), 
                          (imshape[1],imshape[0])]], 
                        dtype=np.int32)

    return region_of_interest(cannied, vertices) 

# Hough transform parameters.
rho = 1             # distance resolution in pixels of the Hough grid
theta = np.pi/180   # angular resolution in radians of the Hough grid
threshold = 40      # minimum number of votes (intersections in Hough grid cell)
min_line_length = 7 # minimum number of pixels making up a line
max_line_gap = 2    # maximum gap in pixels between connectable line segments

def draw_hough_lines(image):
    narrowed = pre_hough(image)
    return hough_lines(narrowed, rho, theta, threshold, min_line_length, max_line_gap)

def convert_to_degrees(radians):
    return radians * 180 / math.pi

class LineClusters:
    def __init__(self, image, name, debug):
        self.image = image
        self.name = name
        self.debug = debug
        [self.lineclusters, 
         self.slopeclusters, 
         self.linkage, 
         self.dmax, 
         self.houghlinecount] = LineClusters.__find_line_clusters(image, debug)
        [self.candidate_lines,
         self.inf_found] = LineClusters.__get_candidate_lines(self.lineclusters, self.slopeclusters)

    @staticmethod
    def __calc_average_slopes(slopeclusters):
        average_lines = []
        for cluster in slopeclusters:
            slope_sum = 0.
            intercept_sum = 0.
            for slope_intercept_pair in cluster:
                slope_sum += slope_intercept_pair[0]
                intercept_sum += slope_intercept_pair[1]
            average_lines.append([slope_sum / len(cluster), intercept_sum / len(cluster)])
        return average_lines
    
    @staticmethod
    def __get_cluster_extents(lineclusters):
        xmin = []
        xmax = []
        ymin = []
        ymax = []
        for cluster in lineclusters:
            xvalues = []
            yvalues = []
            for line in cluster:
                for x1,y1,x2,y2 in line:
                    xvalues.append(x1)
                    xvalues.append(x2)
                    yvalues.append(y1)
                    yvalues.append(y2)
            xmin.append(min(xvalues))
            xmax.append(max(xvalues))
            ymin.append(min(yvalues))
            ymax.append(max(yvalues))
        return [xmin, xmax, ymin, ymax]
    
    @staticmethod
    def __get_candidate_lines(lineclusters, slopeclusters):
        average_lines = LineClusters.__calc_average_slopes(slopeclusters)
        [xmin, xmax, ymin, ymax] = LineClusters.__get_cluster_extents(lineclusters)
        
        candidate_lines = []
        inf_found = False
        
        for i in range(len(average_lines)):
            m_and_b = average_lines[i]
            m = m_and_b[0]
            b = m_and_b[1]
            if math.isfinite(m) and math.isfinite(b):
                x1 = xmin[i]
                x2 = xmax[i]
                # Equation of a line: y = mx + b
                y1 = int(m * x1 + b)
                y2 = int(m * x2 + b)
            else:
                x1 = xmin[i]
                x2 = xmax[i]
                y1 = ymin[i]
                y2 = ymax[i]
                inf_found = True

            candidate_lines.append([[[x1, y1, x2, y2]]])
        return [candidate_lines, inf_found]
    
    @staticmethod
    def __filter_lines_by_angle(image):
        narrowed = pre_hough(image)
        lines = get_hough_lines(narrowed, rho, theta, threshold, min_line_length, max_line_gap)
        angles = LineClusters.__calc_angle(lines)
        
        deletions = []
        for i in range(len(angles)-1, -1, -1):
            angle = angles[i]
            if ((angle > 175) or (angle < 95 and angle > 85) or (angle < 5)):
                deletions.append(i)
                angles.pop(i)
        
        filtered_lines = np.delete(lines, deletions, axis=0)
        
        return [filtered_lines, angles]
    
    @staticmethod
    def __find_line_clusters(image, debug):        
        [lines, angles] = LineClusters.__filter_lines_by_angle(image)
        
        slopes = LineClusters.__calc_slope_and_intercept(lines)
        
        angles_and_intercepts = []
        for i in range(len(slopes)):
            angles_and_intercepts.append([angles[i], slopes[i][1]])

        # Use hierarchical clustering to cluster lines based on angle and intercept.
        d = dist.pdist(angles_and_intercepts, metric='euclidean')
        linkage = hier.linkage(d, method='single')

        distance_threshold = 30
        clusters = hier.fcluster(linkage,
                                 # threshold_max_fraction * d.max(), 
                                 distance_threshold,
                                 'distance')

        # Separate lines into clusters.
        lineclusters = []
        slopeclusters = []
        angleclusters = []
        for i in range(len(clusters)):
            cluster = clusters[i]
            while (cluster > len(lineclusters)):
                lineclusters.append([])
                slopeclusters.append([])
                angleclusters.append([])
            lineclusters[cluster - 1].append(lines[i])
            slopeclusters[cluster - 1].append(slopes[i])
            angleclusters[cluster - 1].append(angles[i])
            
        return [lineclusters, slopeclusters, linkage, d.max(), len(lines)]

    @staticmethod
    def __calc_slope_and_intercept(lines):
        lines_with_slope = []
        for line in lines:
            for x1,y1,x2,y2 in line:
                m = (y2 - y1) / (x2 - x1)
                b = y1 - m * x1
                lines_with_slope.append([m, b])
        return lines_with_slope
    
    @staticmethod
    def __calc_angle(lines):
        angles = []
        for line in lines:
            for x1,y1,x2,y2 in line:
                x = []
                y = []
                x.append(x1)
                x.append(x2)
                y.append(y1)
                y.append(y2)
                if (x1 < x2):
                    leftmost = 0
                else:
                    leftmost = 1
                adjacent = x[1-leftmost] - x[leftmost]
                negative = (y[1-leftmost] - y[leftmost]) < 0
                hypotenuse = math.sqrt(math.pow((x2-x1), 2) + math.pow((y2-y1), 2))
                if negative:
                    hypotenuse = -hypotenuse
                radians = math.acos(adjacent / hypotenuse)
                degrees = convert_to_degrees(radians)
                angles.append(degrees)
        return angles
    
    def __draw_clusters_on_black(self, clusters, thickness):
        # Draw line clusters with different colors.
        line_img = np.zeros((self.image.shape[0], self.image.shape[1], 3), dtype=np.uint8)

        # Red is special. It means we ran out of colors.
        red = [255, 0, 0]
        colors = [[64, 0, 255],
                  [0, 0, 255],
                  [0, 255, 255],
                  [0, 255, 0],
                  [128, 255, 0],
                  [255, 255, 0],
                  [255, 128, 0],
                  red
                 ]

        color_i = 0
        for lines in clusters:
            draw_lines(line_img, lines, colors[color_i], thickness)
            color_i += 1
            if (color_i == len(colors)):
                color_i = len(colors) - 1
                
        return line_img

    def get_blended_image(self, draw_candidates):
        if (draw_candidates):
            clusters = self.candidate_lines
            thickness = 10
        else:
            clusters = self.lineclusters
            thickness = 2
        
        drawn_lines = self.__draw_clusters_on_black(clusters, thickness)
        
        return weighted_img(drawn_lines, self.image)
    
    def __make_title(self, id = ''):
        if id:
            id = ', ' + id
        return (self.name
                + id
                + ', dmax=' + "{0:.1f}".format(self.dmax) 
                + ', clusters=' + str(len(self.lineclusters)) 
                + ', houghlines=' + str(self.houghlinecount))
    
    def plot_dendrogram(self, fignum):
        plt.figure(fignum)
        plt.title(self.__make_title())
        hier.dendrogram(self.linkage)
        
    def draw_image(self, fignum, draw_candidates, id = ''):
        plt.figure(fignum)
        if id:
            id += ', '
        if (draw_candidates):
            id += 'candidate'
        else:
            id += 'lineclusters'
        plt.title(self.__make_title(id))
        blended = self.get_blended_image(draw_candidates)
        print_image(blended, grayscale = False)
    
def hough_only(image):
    drawn_lines = draw_hough_lines(image)
    blended = weighted_img(drawn_lines, image)
    return blended

import os

try_cluster = True
path = "test_images/"
resultPrefix = "result."
files = os.listdir(path)
i = 1
for filename in files:
    if (filename.startswith(resultPrefix)):
        continue
    image = mpimg.imread(path + filename)
    
    if (try_cluster):
        clusters = LineClusters(image, filename, debug=True)
        
        clusters.plot_dendrogram(i)
        i += 1
        clusters.draw_image(i, draw_candidates = True)
        i += 1
        clusters.draw_image(i, draw_candidates = False)
    else:
        plt.figure(i)
        blended = hough_only(image)
        print_image(blended, grayscale = True)

    plt.savefig(path + resultPrefix + filename, bbox_inches='tight')
    i += 1

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
# Download ffmpeg.
import imageio
imageio.plugins.ffmpeg.download()

In [ ]:
# Import everything needed to edit/save/watch video clips
import moviepy
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
frame = 1
fignum = 0
dmax_threshold = 1000

def process_image(image):
    # The output should be a color image (3 channel) for processing video below.
    global frame, fignum
    name = 'frame=' + str(frame)
    clusters = LineClusters(image, name, debug=False)
    
    debug_reason = ''
    if clusters.dmax > dmax_threshold:
        debug_reason = 'dmax'
    if clusters.inf_found:
        debug_reason = 'inf'
    
    if debug_reason:
        #fignum += 1
        #clusters.plot_dendrogram(fignum)
        fignum += 1
        clusters.draw_image(fignum, draw_candidates = True, id = debug_reason)
        fignum += 1
        clusters.draw_image(fignum, draw_candidates = False, id = debug_reason)
        
    frame += 1
    return clusters.get_blended_image(draw_candidates = True)

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!


## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))